# 第8章　実践編3: シングルセル解析とVAE

- 水越周良
- 小嶋泰弘
- 島村徹平

編集部注：2023年5月29日最終更新．コードの一部がお手元の書籍と異なる可能性がございます．正誤・更新情報は弊社ウェブサイトの[本書詳細ページ](https://www.yodosha.co.jp/jikkenigaku/book/9784758122634/index.html)をご参照ください．

##### 入力8-1


In [ ]:
!pip install scanpy==1.9.3 umap-learn==0.5.3 leidenalg==0.9.1 pyro-ppl==1.8.4 scvi==0.6.8

##### 入力8-2


In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import scanpy as sc
import torch
import torch.nn as nn
import torch.distributions as dist
from torch.nn.parameter import Parameter
from torch import functional as F
from torch.distributions.kl import kl_divergence
from torch.nn import init
from torch.utils.data import DataLoader
import copy
import os
import sys
from matplotlib import pyplot as plt
import scipy
import umap
import anndata as ad
import sklearn.decomposition
from sklearn.metrics.cluster import adjusted_rand_score
import pyro
import scvi
import scvi.dataset
from scvi.dataset import CortexDataset # RetinaDatasetやPbmcDatasetにもクラスタのラベルがある

##### 入力8-3


In [ ]:
import warnings; warnings.simplefilter('ignore') # 警告メッセージを表示しない
cortex = CortexDataset(save_path='data/', total_genes=None) # scVIのデータセットをロードする
n_genes = 1000 # VAEに入れる遺伝子の数を設定する. 多くの細胞を含むデータの際はより多くの遺伝子を用いたほうが良い
cortex.subsample_genes(n_genes, mode="variance") # 細胞間の分散が高い順から遺伝子を選ぶ
adata = ad.AnnData(cortex.X) # anndataにデータを格納
adata.var_names = cortex.gene_names # 遺伝子名を格納
adata.obs_names = [f'{i}' for i in range (adata.n_obs)]
adata.obs['cluster_labels'] = cortex.labels.astype('str') # データセットに付属のクラスタラベルを格納
adata.layers['raw_counts'] = adata.X.astype(int) # VAEのモデルでは正規化前の発現量を用いるために別の場所に格納する
sc.pp.filter_cells(adata, min_genes=200) # 発現する遺伝子が少なすぎる細胞を除く
sc.pp.normalize_total(adata, target_sum=1e4) # 正規化
sc.pp.log1p(adata) # 対数変換

##### 入力8-4


In [ ]:
adata

##### 入力8-5


In [ ]:
class LinearReLU(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearReLU, self).__init__()
        self.f = nn.Sequential(
            nn.Linear(input_dim, output_dim), # 線形変換
            nn.BatchNorm1d(output_dim), # バッチ正規化
            nn.ReLU(True)) # 活性化関数
        
    def forward(self, x):
        h = self.f(x)
        return(h)

class Encoder(nn.Module):
    def __init__(self, z_dim, h_dim, n_genes):
        super(Encoder, self).__init__()
        self.x2h = LinearReLU(n_genes, h_dim)
        self.seq_nn = LinearReLU(h_dim, h_dim)
        self.h2mu = nn.Linear(h_dim, z_dim) # zの従う正規分布の平均を出力
        self.h2logvar = nn.Linear(h_dim, z_dim) # zの従う正規分布の分散を出力

    def forward(self, x):
        pre_h = self.x2h(x)
        post_h = self.seq_nn(pre_h)
        mu = self.h2mu(post_h)
        logvar = self.h2logvar(post_h)
        return(mu, logvar)
    
class Decoder(nn.Module):
    def __init__(self, z_dim, h_dim, n_genes):
        super(Decoder, self).__init__()
        self.z2h = LinearReLU(z_dim, h_dim)
        self.seq_nn = LinearReLU(h_dim, h_dim)
        self.h2ld = nn.Linear(h_dim, n_genes)
        self.softplus = nn.Softplus()
    
    def forward(self, z):
        pre_h = self.z2h(z)
        post_h = self.seq_nn(pre_h)
        ld = self.h2ld(post_h)
        correct_ld = self.softplus(ld) # 出力が正の値(特に0から1の間)になるようにsoftplus変換する
        return(correct_ld)

##### 入力8-6


In [ ]:
def calcNbLoss(ld, norm_mat, r, obs): # デコーダの分布が負の二項分布に従うと仮定した場合の対数尤度を求める
    ld = norm_mat * ld
    p = ld / (ld + r)
    p_z = dist.NegativeBinomial(r, p)
    l = - p_z.log_prob(obs) # 対数尤度を求める
    return(l)

def calcZeroInflatedNbLoss(ld, norm_mat, r, obs, gate): # デコーダの分布がゼロ過剰負の二項分布に従うと仮定した場合の対数尤度を求める
    ld = norm_mat * ld
    p = ld / (ld + r)
    p_z = pyro.distributions.zero_inflated.ZeroInflatedDistribution(base_dist=dist. NegativeBinomial(r, p),gate=gate)
    l = - p_z.log_prob(obs) # 対数尤度を求める
    return(l)

def calcPoissonLoss(ld, norm_mat, obs): # デコーダの分布がポアソン分布に従うと仮定した場合の対数尤度を求める
    p_z = dist.Poisson(ld * norm_mat)
    l = - p_z.log_prob(obs) # 対数尤度を求める
    return(l)

def calcZeroInflatedPoissonLoss(ld, norm_mat, obs, gate): # デコーダの分布がゼロ過剰ポアソン分布に従うと仮定した場合の対数尤度を求める
    p_z = pyro.distributions.zero_inflated.ZeroInflatedDistribution(base_dist=dist.Poisson(ld * norm_mat), gate=gate)
    l = - p_z.log_prob(obs) # 対数尤度を求める
    return(l)

##### 入力8-7


In [ ]:
class VAE(nn.Module):
    def __init__(self, z_dim, h_dim, n_genes, likelihood_function='zero-inflated_negative_binominal'):
        super(VAE, self).__init__()
        self.r = Parameter(torch.Tensor(n_genes)) # 負の二項分布に必要なパラメータ
        self.drop_rate = Parameter(torch.Tensor(n_genes)) # ゼロ過剰分布に必要なパラメータ
        self.enc_z = Encoder(z_dim, h_dim, n_genes) # エンコーダをインスタンス化
        self.dec_z = Decoder(z_dim, h_dim, n_genes) # デコーダをインスタンス化
        self.softplus = nn.Softplus() # softplus関数. 値を正の値に変換する. ReLUと比べ，入力が負の値でもパラメータの更新が行われるメリットがある
        self.sigmoid = nn.Sigmoid() # シグモイド関数. 値を0から1の間の値に変換する
        self.likelihood_function = likelihood_function
        self.reset_parameters() # 学習開始時にパラメータをリセットする

    def reset_parameters(self):
        init.normal_(self.r)
        init.normal_(self.drop_rate)

    def forward(self, x):
        qz_mu, qz_logvar = self.enc_z(x) # エンコーダからzの平均と分散を出力
        qz = dist.Normal(qz_mu, self.softplus(qz_logvar)) # zの従う正規分布を作る
        z = qz.rsample() # reparameterization trickにより逆伝播できるようになる
        x_hat = self.dec_z(z) # デコーダからの出力
        return(qz, x_hat)

    def elbo_loss(self, x, norm_mat):
        qz, x_hat = self(x)
        kld = 0.5 * (qz.loc.pow(2) + qz.scale.pow(2) - 1 - qz.scale.pow(2).log()) # KLダイバ ージェンス
        if self.likelihood_function == 'zero-inflated_negative_binominal':
        # rは負の二項分布における失敗の回数と同義であり，ポアソン分布からの離れ具合を制御する. rが無限大の時，負の二項分布はポアソン分布と一致する
            r = self.softplus(self.r) # 0より大きい値を取るために，softplusで変換する
            drop_rate = self.sigmoid(self.drop_rate) # ゼロ過剰分布において，サンプリングされた値が0になる割合. シグモイド関数で0から1の値にする
            lx = calcZeroInflatedNbLoss(x_hat, norm_mat, r, x, drop_rate) # デコーダの分布の対数尤度を求める
        if self.likelihood_function == 'negative_binominal':
            r = self.softplus(self.r)
            lx = calcNbLoss(x_hat, norm_mat, r, x)
        if self.likelihood_function == 'zero-inflated_poisson':
            drop_rate = self.sigmoid(self.drop_rate)
            lx = calcZeroInflatedPoissonLoss(x_hat, norm_mat, x, drop_rate)
        if self.likelihood_function == 'poisson':
            lx = calcPoissonLoss(x_hat, norm_mat, x)
        elbo_loss = torch.sum((torch.sum(kld, dim=-1))) + torch.sum((torch.sum(lx, dim=-1)))
        return(elbo_loss)

##### 入力8-8


In [ ]:
class EarlyStopping:
    def __init__(self, patience=10, path='checkpoint.pt'): # 何回更新が止まったら学習を止めるのか設定
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.stop_flag = False # これがTrueになったら学習を止める
        self.path = path

    def __call__(self, validation_loss, model):
        score = validation_loss
        if self.best_score is None:
            self.best_score = score # 1エポック目はそのままスコアを保存
            self.checkpoint(validation_loss, model)
        elif score > self.best_score: # 損失が更新されなかったらカウンタに1を加える
            self.counter += 1
            if self.counter >= self.patience: # 設定カウントになったら学習を止める
                self.stop_flag = True
        else: # 損失が減った場合
            self.best_score = score # その損失をベストスコアにする
            self.checkpoint(validation_loss, model) # モデルを保存
            self.counter = 0 # カウンタをリセット

    def checkpoint(self, validation_loss, model):
        torch.save(model.state_dict(), self.path) # パラメータをpathに保存

##### 入力8-9


In [ ]:
class DataSet(torch.utils.data.Dataset):
    def __init__(self, x, norm_mat, transform=None, pre_transform=None):
        self.x = x
        self.norm_mat = norm_mat

    def __len__(self):
        return(self.x.shape[0]) # 全細胞数を返す
    
    def __getitem__(self, idx):
        idx_x = self.x[idx]
        idx_norm_mat = self.norm_mat[idx]
        return(idx_x, idx_norm_mat) # 与えられたindexに対応するxとnorm_matを返す

class DataManager():
    def __init__(self, x, test_ratio=0.05, batch_size=100, num_workers=1, validation_ratio=0.1):
        x = x.float()
        # デコーダからの出力の平均が1になるような行列を作る
        norm_mat = torch.sum(x, dim=1).view(-1, 1) * torch.sum(x, dim=0).view(1, -1)
        norm_mat = torch.mean(x) * norm_mat / torch.mean(norm_mat)
        self.x = x
        self.norm_mat = norm_mat
        total_num = x.shape[0] # 全細胞数
        validation_num = int(total_num * validation_ratio) # validationに使う細胞数
        test_num = int(total_num * test_ratio) # testに使う細胞数
        np.random.seed(42) # シード値を指定
        idx = np.random.permutation(np.arange(total_num)) # 細胞のインデックスをシャッフルし， validation，test，trainingに使われる細胞のインデックスを得る
        validation_idx, test_idx, train_idx = idx[:validation_num], idx[validation_num:(validation_num + test_num)], idx[(validation_num + test_num):]
        self.validation_idx, self.test_idx, self.train_idx = validation_idx, test_idx, train_idx
        self.validation_x = x[validation_idx]
        self.validation_norm_mat = norm_mat[validation_idx]
        self.test_x = x[test_idx]
        self.test_norm_mat = norm_mat[test_idx]
        self.train_eds = DataSet(x[train_idx], norm_mat[train_idx]) # trainingに用いる細胞を格納
        # DataLoaderはDataSetから受け取ったデータをミニバッチごと(今回は100細胞ごと)に分けて出力してくれる
        self.train_loader = torch.utils.data.DataLoader(self.train_eds, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)

##### 入力8-10


In [ ]:
class Experiment:
    def __init__(self, lr, x, z_dim, h_dim,n_genes, likelihood_function):
        self.dm = DataManager(x) # DataManagerクラスをインスタンス化
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 可能ならGPUを使用
        self.model = VAE(z_dim, h_dim, n_genes, likelihood_function) # VAEクラスをインスタンス化
        self.model.to(self.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr) # OptimizerにはAdamを使用
        self.train_loss_list = [] # training lossを記録する
        self.val_loss_list = [] # validation lossを記録する

    def train_epoch(self): # trainingの実行
        self.model.train() # trainingモードにする
        total_loss = 0
        entry_num = 0
        for x, norm_mat in self.dm.train_loader: # 学習用データをバッチごとに出力してくれる
            x = x.to(self.device) # 100細胞の発現量が取り出される
            norm_mat = norm_mat.to(self.device) # 取り出された細胞に対応するnorm_mat
            self.optimizer.zero_grad() # 前回計算した勾配がoptimizerに残っているため初期化する
            loss = self.model.elbo_loss(x, norm_mat) # ELBOを計算する
            loss.backward() # 誤差逆伝播を行う
            self.optimizer.step() # パラメータの最適化を行う
            total_loss += loss.item() # バッチごとにlossが計算されるのでそれを足していく
            entry_num += x.shape[0] # バッチごとに細胞数を足していく
        return(total_loss / entry_num) # すべてのバッチでのlossの合計を細胞数で割ったものが出力される
    
    def evaluate(self): # validation lossを計算
        self.model.eval() # validationとtest時にはこのように書く
        x = self.dm.validation_x.to(self.device) # validation用のxを取り出す
        norm_mat = self.dm.validation_norm_mat.to(self.device) # 取り出したxに対応するnorm_mat
        
        loss = self.model.elbo_loss(x, norm_mat)
        entry_num = x.shape[0]
        loss_val = loss / entry_num
        return(loss_val) # lossの値を出力
    
    def test(self): # test_lossを計算
        self.model.eval()
        x = self.dm.test_x.to(self.device) # test用のxを取り出す
        norm_mat = self.dm.test_norm_mat.to(self.device) # 取り出したxに対応するnorm_mat
        loss = self.model.elbo_loss(x, norm_mat)
        entry_num = x.shape[0]
        loss_val = loss / entry_num
        return(loss_val)
    
    def train_total(self, epoch_num): # trainingの回数や終了を管理する
        earlystopping = EarlyStopping() # EarlyStoppingクラスをインスタンス化する
        for epoch in range(epoch_num): # 指定された回数，学習を行う
            loss = self.train_epoch() # trainingを行うtrain_epoch()を呼び出す
            val_loss = self.evaluate() # validation lossの計算
            self.train_loss_list.append(loss) # listにtraining lossを格納
            self.val_loss_list.append(val_loss) # listにvalidation lossを格納
            if epoch % 20 == 0:
                print(f'validation loss at epoch {epoch} is {val_loss:.2f}') # 20回ごとに validation lossを出力
            earlystopping(val_loss, self.model) # validation lossの増減を判断
            if earlystopping.stop_flag: # ストップフラグがTrueの場合，breakでtrainingを終える
                print(f'Early Stopping at {epoch} epoch')
                break

##### 入力8-11


In [ ]:
lr = 0.001 # 学習率を設定
if type(adata.layers['raw_counts']) == np.ndarray: # 入力をndarrayからtensorに変換
    x = torch.tensor(adata.layers['raw_counts'].copy())
else: # 疎行列の場合はndarrayに変換してから，tensorに変換
    x = torch.tensor(adata.layers['raw_counts'].toarray().copy())
z_dim = 10 # zの次元数
h_dim = 128 # 隠れ層の次元数
vae_exp = Experiment(lr, x, z_dim, h_dim, n_genes, likelihood_function='zero-inflated_negative_binominal')
print('Start!')
vae_exp.train_total(500) # 学習の実行
vae_exp.model.load_state_dict(torch.load('checkpoint.pt')) # 最後に保存したモデルを読み込む
print(f'Done! validation_loss:{vae_exp.evaluate():.2f}')
# 学習曲線を描こう
val = [i.item() for i in vae_exp.val_loss_list]
train = vae_exp.train_loss_list
xx = [i for i in range(len(val))]
fig = plt.figure()
ax = fig.add_subplot()
ax.plot(xx, val, label='validation_loss')
ax.plot(xx, train, label='training_loss')
ax.legend()
fig.show()


##### 入力8-12


In [ ]:
x = vae_exp.dm.x # 今回用いた細胞を取り出す
qz_mu, qz_logvar = vae_exp.model.enc_z(x) # 学習後のエンコーダから平均と分散を得る
qz = dist.Normal(qz_mu, vae_exp.model.softplus(qz_logvar))
z_vae = qz.loc # 学習を終えた後は，zをランダムサンプリングで求めるのではなく，平均の値を用いる
adata.obsm['z_vae'] = z_vae.to('cpu').detach().numpy().copy()
sc.pp.neighbors(adata, n_neighbors=20, n_pcs=z_dim, use_rep='z_vae', key_added='z_vae') # z_vaeの値で近傍グラフを作成
sc.tl.umap(adata, neighbors_key='z_vae')
sc.tl.leiden(adata, neighbors_key='z_vae', key_added='VAE_clusters') # z_vaeの値でクラスタリング
sc.pl.umap(adata, color=['VAE_clusters', 'cluster_labels'], wspace=0.3) # umapで図示

train_counts = adata.X[vae_exp.dm.train_idx] # 評価のため，PCAのモデルの学習にもtrainingに含まれる細胞のみを用いる
pca_model = sklearn.decomposition.PCA(n_components=z_dim) # zの次元数と同じ次元まで削減
pca_model.fit(train_counts) # PCAのモデルの学習
z_pca = pca_model.transform(adata.X) # 学習したモデルですべての細胞を次元削減
adata.obsm['z_pca'] = z_pca
sc.pp.neighbors(adata, n_neighbors=20, n_pcs=z_dim, use_rep='z_pca', key_added='z_pca') # z_pcaの値で近傍グラフを作成
sc.tl.umap(adata, neighbors_key='z_pca')
sc.tl.leiden(adata, neighbors_key='z_pca', key_added='PCA_clusters') # z_pcaの値でクラスタリング
sc.pl.umap(adata, color=['PCA_clusters','cluster_labels'], wspace=0.3)

clusters_label = adata.obs['cluster_labels'].values.tolist() # データセットのラベル
pca_clusters_label = adata.obs['PCA_clusters'].values.tolist()
vae_clusters_label = adata.obs['VAE_clusters'].values.tolist()
# ARIで与えられたクラスタとの類似度を見る
print('\n',f'ARI score : clusters_label vs PCA_clusters {adjusted_rand_score(clusters_label, pca_clusters_label):.3f}')
print((f'ARI score : clusters_label vs VAE_clusters {adjusted_rand_score(clusters_label, vae_clusters_label):.3f}'))